# Value + error

Use valuation model and error model in order to determine when to trade

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
import seaborn as sns
import matplotlib.pyplot as plt
from keras import layers
import pandas_datareader as pdr
from datetime import datetime
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
import itertools
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform


Using TensorFlow backend.


In [2]:

NUM_INPUT_NEURONS = 64
NUM_OUTPUT_NEURONS = 1
DAY_OFFSET = 5

def from_network(symbol):
    return pdr.get_data_yahoo(symbols=symbol, start=datetime(1900, 1, 1)).sort_values(by=['Date'],ascending=False)

# Create features (only close price for now)
def convert_to_percentage(old, new):
    return (old - new) / old

def convert_labels_to_category(labels): 
    # Simplification - If positive return, 1, else 0
    # return map(lambda arr: 1 if arr[0] > 1 else 0, labels)
    # rounding simpliciation
    return map(lambda arr: map(lambda val: round(val,4),arr), labels)

def convert_to_train(raw_dataset, offset=5):
    dataset = raw_dataset.copy()
    features = []
    labels = []
    for i in range(offset, len(dataset) - NUM_INPUT_NEURONS):

        feature_dataset = dataset[i:i+NUM_INPUT_NEURONS].copy()
        latest_close = feature_dataset['Close'].iloc[0]
        
        features.append(
            feature_dataset['Close']
                .map(lambda current: convert_to_percentage(latest_close, current))
                .tolist()
        )
        labels.append([
            dataset['Close'].iloc[i-1] / latest_close, # 1 day trade
        ])
        
    # Without converting labels the precision is hard to determine accuracy. 
    # Rather than crude 0/1, maybe this can be more sophisticated
    labels = convert_labels_to_category(labels)
    
    return [features,labels]

def split_data(symbol):
    fetched = from_network(symbol)
    converted = convert_to_train(fetched)
    features = converted[0]
    labels = converted[1]
    training = [ # since we decide a model, use all data for training
        features[1:],
        labels[1:]
    ]
    validation = [
        features[5:1000],
        labels[5:1000]
    ]
    prediction = [
        features[:5],
        labels[:5],
    ]
    
    return {
        'symbol': symbol,
        'prediction': prediction,
        'validation': validation,
        'training': training,
    }

    
def combine_all(accum, prep):
    return {   
        'prediction':[
            accum['prediction'][0] + prep['prediction'][0],
            accum['prediction'][1] + prep['prediction'][1],
        ],
        'validation':[
            accum['validation'][0] + prep['validation'][0],
            accum['validation'][1] + prep['validation'][1],
        ],
        'training':[
            accum['training'][0] + prep['training'][0],
            accum['training'][1] + prep['training'][1],
        ],
    }



In [14]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
    error_model = load_model('sector_model_error.h5')
    valuation_model = load_model('sector_model.h5')



In [15]:
prepped_data = map(split_data, [
    'QQQ',
    'SPY',
    'IWM',
    'XLK',
    'XLF',
    'XLE',
    'XLP',
    'XLV',
    'XLY',
    'XLI',
    'XLU',
])


In [16]:
for i in range(len(prepped_data)):
    print prepped_data[i]['symbol']
combined = reduce(combine_all, prepped_data,{
    'prediction':[[],[]],
    'validation':[[],[]],
    'training':[[],[]],
})

#len(converted_feature_set[0][0])
# print len(combined['training'][0])
# train_data = np.array(combined['training'][0])
# train_labels = np.array(combined['training'][1])

print len(combined['validation'][0])
validation_data = np.array(combined['validation'][0])
validation_labels = np.array(combined['validation'][1])

# print len(combined['prediction'][0])
# prediction_data = np.array(combined['prediction'][0])
# prediction_labels = np.array(combined['prediction'][1])


QQQ
SPY
IWM
XLK
XLF
XLE
XLP
XLV
XLY
XLI
XLU
10945


In [17]:
value_predictions = valuation_model.predict(validation_data)
error_data = np.array(map(lambda inputs: list(itertools.chain(inputs[0],inputs[1])), zip(validation_data, validation_labels)))
error_predictions = error_model.predict(error_data)




In [18]:
# value_predictions

In [19]:
# error_predictions

In [20]:
# validation_labels

In [21]:
# signal using error term modifier with valuation
signals = map(lambda arr: (arr[0] - arr[1])[0],zip(value_predictions, error_predictions))

# signal using valuation only
# signals = map(lambda arr: (arr[0])[0],zip(value_predictions, error_predictions))

trades = map(lambda signal: 1 if round(signal,4) > 1 else 0,signals)

In [28]:
# one day return
actual = map(lambda arr: arr[0],validation_labels)

df = pd.DataFrame({
    'valuation': value_predictions.flatten(),
    'error': error_predictions.flatten(),
    'signal':signals,
    'actual': actual,
    'trade':trades,
})
df['entry_success'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 1) else 0,axis=1)
df['entry_failure'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 1) else 0,axis=1)
df['avoid_success'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 0) else 0,axis=1)
df['avoid_failure'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 0) else 0,axis=1)

# primitive policy - replace with a policy network which maximizes reward
def label_success (row):
    return 0 if (row['entry_failure'] == 1 or row['entry_failure'] == 1) else 1

success = df.apply (lambda row: label_success (row),axis=1)
df['success'] = success;
df

,actual,error,signal,trade,valuation,entry_success,entry_failure,avoid_success,avoid_failure,success
0,0.9876,-0.006930,1.006959,1,1.000029,0,1,0,0,0
1,1.0119,-0.018403,1.021104,1,1.002701,1,0,0,0,1
2,1.0065,-0.010895,1.013789,1,1.002894,1,0,0,0,1
3,1.0013,-0.001447,1.001735,1,1.000287,1,0,0,0,1
4,0.9800,0.007764,0.993440,0,1.001204,0,0,1,0,1
5,1.0099,-0.004930,1.004745,1,0.999815,1,0,0,0,1
6,1.0079,-0.010204,1.011465,1,1.001262,1,0,0,0,1
7,0.9998,-0.004861,1.005003,1,1.000142,0,1,0,0,0
8,1.0195,-0.008092,1.007105,1,0.999013,1,0,0,0,1
9,0.9912,0.013892,0.987566,0,1.001458,0,0,1,0,1


In [29]:
print df['actual'].corr(df['signal'])
print df['actual'].corr(df['trade'])
print df['actual'].corr(df['entry_success'])
print df['actual'].corr(df['success'])

0.7729687066459356
0.5446092842436379
0.6487210133072515
0.14927042349122813


In [30]:
NUM_TEST_SAMPLES = len(validation_data)
print NUM_TEST_SAMPLES

print '\nPrecision' # optimize for this since we can increase discovery, so long as we find enough trades
print sum(df['entry_success']) * 1.00 / (sum(df['entry_success']) + sum(df['entry_failure'])) 

print '\nRecall'
print sum(df['entry_success']) * 1.00 / (sum(df['entry_success']) + sum(df['avoid_failure']))

print '\nAccuracy'
print sum(df['entry_success']) * 1.00 / (NUM_TEST_SAMPLES)

print '\nNon-loss events'
print sum(df['success'])
print sum(df['success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nLose trades'
print sum(df['entry_failure'])
print sum(df['entry_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nWin trades'
print sum(df['entry_success'])
print sum(df['entry_success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nMissed opportunities'
print sum(df['avoid_failure'])
print sum(df['avoid_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nBullets dodged'
print sum(df['avoid_success'])
print sum(df['avoid_success']) / (NUM_TEST_SAMPLES * 1.00)


10945

Precision
0.762324518446

Recall
0.803509979353

Accuracy
0.426678848789

Non-loss events
9489
0.866971219735

Lose trades
1456
0.133028780265

Win trades
4670
0.426678848789

Missed opportunities
1142
0.104339881224

Bullets dodged
3586
0.327638190955
